
# Segmenting and Clustering Neighborhoods in Toronto

### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
#import modules
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [2]:
# import data using url provided
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url_text=requests.get(url).text
#url_text

### Use BeautifulSoup to list the webpage and extract the table out of the webpage
The first row the page is assigned initially using 'Not assigned', which will be removed in the next step.

In [3]:
# using BeautifulSoup to process url_text
city_data = BeautifulSoup(url_text, 'html.parser')

# creat a new Dataframe
column_names = ['PostalCode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = city_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 'Not assigned'
borough = 'Not assigned'
neighborhood = 'Not assigned'

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n')
    toronto = toronto.append({'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

print('The shape of the original table is ',toronto.shape)
toronto.head()

The shape of the original table is  (181, 3)


,PostalCode,Borough,Neighborhood
0,Not assigned,Not assigned,Not assigned
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### Remove all rows with 'Not assigned'.
### List the head of the dataframe and print out the shape of the dataframe

In [4]:
# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto.reset_index(drop = True, inplace = True)

print('The shape of the cleaned table is ',toronto.shape)
toronto.head()

The shape of the cleaned table is  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Use the csv file to create the following dataframe

In [5]:
!wget -O geospatial_data.csv https://cocl.us/Geospatial_data
geocsv_data = pd.read_csv('geospatial_data.csv')
geocsv_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geocsv_data.head()

--2020-08-01 08:50:17--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.69.24, 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-01 08:50:19--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-01 08:50:19--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [f

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge dataframe totonto and geocsv to create a new dataframe

In [6]:
df = pd.merge(toronto, geocsv_data, on='PostalCode')
print('The shape of the cleaned table is ',df.shape)
df.head()

The shape of the cleaned table is  (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
